In [1]:
from pgtools import maf_parser
from pgtools.gff_parser import parse_GFFs_dir, Gff, parse_gff
from pgtools.utils import intersection_len
from itertools import combinations
from copy import deepcopy
from pgtools.pangenome import Pangenome
import os

In [2]:
maf_dir = "/home/pampuch/studia/magisterka/test_data/klebsiella_subset_old/panaroo_new.maf"

In [3]:
maf = maf_parser.parse_maf(maf_dir)
maf = maf.convert_coords_system("gff")

In [4]:
maf = maf_parser.parse_maf(maf_dir)


In [4]:
overlapping_blocks = []
all_sequences = []
for block in list(maf.seq_collections)[:500]:
    # print(block.id)
    for seq in block.sequences:
        all_sequences.append(seq)
        # print(seq.annotation_ids)
        # print(seq.cluster_id)
    # break

In [10]:
block = list(maf.seq_collections)[0]
print(block.sequences)

{<pgtools.maf_parser.MAFseq object at 0x7f3a94a000a0>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a000d0>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a00100>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a01240>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a01270>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a00a90>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a012a0>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a00ac0>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a012d0>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a00af0>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a01300>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a01330>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a01360>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a00b80>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a01390>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a013c0>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a013f0>, <pgtools.maf_parser.MAFseq object at 0x7f3a94a01420>, <pgtools.maf_parser.MAFseq 

1. Take care of sequences overlapping within each block

In [82]:
overlaping_seqs = []
blocks_dict = maf.get_seq_collections_dict()
for s1, s2 in combinations(blocks_dict[4541].sequences,2):
    if s1.seq_name != s2.seq_name:
        continue
    inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    if inter_len > 0:
        overlaping_seqs.append((s1,s2))

In [83]:
overlaping_seqs

[(<pgtools.maf_parser.MAFseq at 0x7f3a97b83820>,
  <pgtools.maf_parser.MAFseq at 0x7f3a97b83850>),
 (<pgtools.maf_parser.MAFseq at 0x7f3a97b83880>,
  <pgtools.maf_parser.MAFseq at 0x7f3a97b838b0>),
 (<pgtools.maf_parser.MAFseq at 0x7f3a97b829e0>,
  <pgtools.maf_parser.MAFseq at 0x7f3a97b82da0>),
 (<pgtools.maf_parser.MAFseq at 0x7f3a97b83580>,
  <pgtools.maf_parser.MAFseq at 0x7f3a97b835e0>),
 (<pgtools.maf_parser.MAFseq at 0x7f3a97b835b0>,
  <pgtools.maf_parser.MAFseq at 0x7f3a97b83610>),
 (<pgtools.maf_parser.MAFseq at 0x7f3a97b82dd0>,
  <pgtools.maf_parser.MAFseq at 0x7f3a97b82e00>),
 (<pgtools.maf_parser.MAFseq at 0x7f3a97b82e30>,
  <pgtools.maf_parser.MAFseq at 0x7f3a97b82e60>),
 (<pgtools.maf_parser.MAFseq at 0x7f3a97b83640>,
  <pgtools.maf_parser.MAFseq at 0x7f3a97b83670>),
 (<pgtools.maf_parser.MAFseq at 0x7f3a97b836a0>,
  <pgtools.maf_parser.MAFseq at 0x7f3a97b836d0>),
 (<pgtools.maf_parser.MAFseq at 0x7f3a97b83700>,
  <pgtools.maf_parser.MAFseq at 0x7f3a97b83730>),
 (<pgtools

In [88]:
overlapping_block_copy = deepcopy(blocks_dict[4541])

In [160]:
def trim_overlap(s1, s2):
    if intersection_len((s1.start, s1.end), (s2.start, s2.end)) == 0:
        # return s1, s2
        return
    else:
        inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    # print(inter_len)
    shorter_seq, longer_seq = sorted([s1, s2], key = lambda x: len(x))
    # shorter_seq = deepcopy(shorter_seq)
    # longer_seq = deepcopy(longer_seq)
    
    # print(shorter_seq)
    # print(longer_seq)
    # there is overlap and we want to trim the shorter sequence: 
    if shorter_seq.start < longer_seq.start:
        # longer seq is begind shorter seq - we want to trimm the end of shorter seq
        shorter_seq.end -= inter_len
    else:
        shorter_seq.start += inter_len

    # return (shorter_seq, longer_seq)
    
def clean_block(block):
    # print(block.id)
    # seqs_to_remove = set()
    # new_seqs = {}
    # overlapping_pairs = {}
    for s1, s2 in combinations(block.sequences,2):
        # print("i")
        if s1.seq_name != s2.seq_name:
            continue
        inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
        if inter_len > 0:
            # print(s1, s2)
            trim_overlap(s1,s2)

            # overlapping_pairs[tuple(sorted((s1.cluster_id, s2.cluster_id)))] = inter_len
            
    # return seqs_to_remove

clean_block(block)

In [122]:
overlapping_block_copy = deepcopy(blocks_dict[4541])
overlapping_block_copy.sequences = list(overlapping_block_copy.sequences)[:7]

In [123]:
clean_block(overlapping_block_copy)

4541
5197_7_4.contig00042: 20451 - 20705, -1 5197_7_4.contig00042: 20702 - 21469, -1
4


In [99]:
for s1, s2 in combinations(overlapping_block_copy.sequences,2):
    if s1.seq_name != s2.seq_name:
        continue
    inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    if inter_len > 0:
        # overlaping_seqs.append((s1,s2))
        print(s1,s2)

In [129]:
a=1
b=2
c=3
list_ = [a,b,c]

for n, i in enumerate(list_):
    print(n)
    list_[n] +=1
    print(list_)
    try:
        print(list_[n-1])
    except:
        continue

0
[2, 2, 3]
3
1
[2, 3, 3]
2
2
[2, 3, 4]
3


In [96]:
a

1

In [69]:
# def trim_overlap(s1, s2):
#     if intersection_len((s1.start, s1.end), (s2.start, s2.end)) == 0:
#         return s1, s2
#     else:
#         inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
#     print(inter_len)
#     shorter_seq, longer_seq = sorted([s1, s2], key = lambda x: len(x))
#     shorter_seq = deepcopy(shorter_seq)
#     longer_seq = deepcopy(longer_seq)
    
#     # print(shorter_seq)
#     # print(longer_seq)
#     # there is overlap and we want to trim the shorter sequence: 
#     if shorter_seq.start < longer_seq.start:
#         # longer seq is begind shorter seq - we want to trimm the end of shorter seq
#         shorter_seq.end -= inter_len
#     else:
#         shorter_seq.start += inter_len

#     return (shorter_seq, longer_seq)

In [68]:
for i in overlaping_seqs[0]: print(i)

5197_2_1.contig00026: 64732 - 65499, 1
5197_2_1.contig00026: 65492 - 65734, 1


In [70]:
trimmed = trim_overlap(overlaping_seqs[0][0], overlaping_seqs[0][1])

8


In [71]:
for i in trimmed: print(i)

5197_2_1.contig00026: 65500 - 65734, 1
5197_2_1.contig00026: 64732 - 65499, 1


## Uproszczenie
!!! Pamiętaj, że sekwencje w bloku muszą mieć tę samą długość (ale to uliniowienie, nie istotne dopóki nie ma w bloku sekwencji. Poprzycięciu sekwencji bloki trzeba by ponownie uliniowić :((( !!!!!!))))

Skoro możesz po prostu zakładać, że przycinasz krótszą sekwencję, to czemu by nie przyciąć po prostu wszystkich występujących overlapów w pliku po parach. Problem - sekwencje mogą rpzestawać na siebie nachodzić w trakcie przycinania z innymi sekwencjami

Dla każdej sekwencji trzeba by wybrac taką co maksymalnie na nią nachodzi na początku i na jej końcu - i co?

### Prosto:
Może zrób tak, że robisz pętle po blokach podwójną i jak znajdziesz overlapy to je przycinasz. Tu jest taki problem, że możesz niepotrzebnie tracić dane - może lepiej porobić pary bloków, policzyć maksymalny overlap i posortować w niego, potem zoribć tą pętlę wg tej kolejności - chyba najlepsza opcja, tylko czy będzie się updatować.

### Finalne:

pary objektów sekwencji posortowane po overlapie. Potem w pętli stosowane trim overlap

#### Pamiętaj

1. Na koniec przywróć do coords z mafa
2. Pozbieraj sekwencje z contigów. !!! Przy - strand trzeba je odpowiednio odwrócić! Ale to masz już gdzieś w kodzie. Czy robisz to ok - możesz porównać z sekwencją sprzed przycięcia
3. jak już masz sekwencje to uliniawianie ich mafftem, przedtem usuń ze wszystkich "-", uliniawiaj tylko te które byly przycinane



In [161]:
all_sequences = []
blocks_subset = deepcopy(list(maf.seq_collections)[:500])
for block in blocks_subset:
    # print(block.id)
    for seq in block.sequences:
        all_sequences.append(seq)
        
overlapping_pairs = {}
for s1, s2 in combinations(all_sequences,2):
    if s1.seq_name != s2.seq_name or s1.strand != s2.strand:
        continue
    inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    if inter_len > 0:
        # print(s1, s2)
        overlapping_pairs[(s1, s2)] = inter_len

In [162]:
sorted_pairs = sorted(list(overlapping_pairs), key=lambda x: overlapping_pairs[x], reverse=True)

In [163]:
print(len(sorted_pairs))

69


In [164]:
for block in blocks_subset:
    clean_block(block)

In [165]:
overlapping_pairs = {}
for s1, s2 in combinations(all_sequences,2):
    if s1.seq_name != s2.seq_name or s1.strand != s2.strand:
        continue
    inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    if inter_len > 0:
        # print(s1, s2)
        overlapping_pairs[(s1, s2)] = inter_len

In [166]:
len(overlapping_pairs)

44

## Now take care of overlaps in between blocks

In [167]:
#check if in fact there are no within block overlaps

for s1, s2 in overlapping_pairs:
    if s1.cluster_id == s2.cluster_id:
        print(s1.cluster_id)

In [169]:
sorted_pairs = sorted(list(overlapping_pairs), key=lambda x: overlapping_pairs[x], reverse=True)

In [171]:
for s1, s2 in sorted_pairs:
    trim_overlap(s1, s2)

In [172]:
overlapping_pairs = {}
for s1, s2 in combinations(all_sequences,2):
    if s1.seq_name != s2.seq_name or s1.strand != s2.strand:
        continue
    inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    if inter_len > 0:
        # print(s1, s2)
        overlapping_pairs[(s1, s2)] = inter_len

In [173]:
overlapping_pairs

{}

In [137]:
"koteczek"[:-3]

'kotec'

In [138]:
"koteczek"[3:]

'eczek'

In [241]:
def trim_overlap(s1, s2):
    if intersection_len((s1.start, s1.end), (s2.start, s2.end)) == 0:
        # return s1, s2
        return
    else:
        inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    # print(inter_len)

    shorter_seq, longer_seq = sorted([s1, s2], key = lambda x: len(x))
    seq_len = len(shorter_seq.seq.replace("-",""))

    old_short = deepcopy(shorter_seq)
    # there is overlap and we want to trim the shorter sequence: 
    if shorter_seq.start < longer_seq.start:
        # longer seq is behind shorter seq - we want to trimm the end of shorter seq
        shorter_seq.end -= (inter_len + 1)
        if shorter_seq.strand > 0:
            shorter_seq.seq = shorter_seq.seq.replace("-","")[:-inter_len - 1]
        else:
            shorter_seq.seq = shorter_seq.seq.replace("-","")[inter_len + 1 :]
    else:
        shorter_seq.start += (inter_len + 1)
        if shorter_seq.strand > 0:
            shorter_seq.seq = shorter_seq.seq.replace("-","")[inter_len + 1:]
        else:
            shorter_seq.seq = shorter_seq.seq.replace("-","")[: -inter_len - 1]
    # return (shorter_seq, longer_seq)
    print("inter", inter_len)
    print(old_short)
    print(old_short.seq)
    print("len", len(old_short.seq.replace("-","")))
    print(shorter_seq)
    print(shorter_seq.seq)
    print("len", len(shorter_seq.seq))
    assert(len(shorter_seq.seq) == len(shorter_seq))
    print("="*40)
    
def clean_block(block):
    for s1, s2 in combinations(block.sequences,2):
        # print("i")
        if s1.seq_name != s2.seq_name:
            continue
        inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
        if inter_len > 0:
            # print(s1, s2)
            trim_overlap(s1,s2)

def trimm_overlaps_maf(pangenome_obj: Pangenome):
    old_coord_sys = pangenome_obj.coord_system
    pangenome_obj = pangenome_obj.convert_coords_system("gff")

    # all_sequences = []
    # for block in pangenome_obj.seq_collections:
    #     # print(block.id)
    #     for seq in block.sequences:
    #         all_sequences.append(seq)
    # overlapping_pairs = {}
    # for s1, s2 in combinations(all_sequences,2):
    #     if s1.seq_name != s2.seq_name or s1.strand != s2.strand:
    #         continue
    #     inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    #     if inter_len > 0:
    #         # print(s1, s2)
    #         overlapping_pairs[(s1, s2)] = inter_len

    # print(len(overlapping_pairs))
    # clean within block overlaps
    for block in pangenome_obj.seq_collections:
        clean_block(block)
        
        
    all_sequences = []
    # blocks_subset = deepcopy(list(maf.seq_collections)[:500])
    for block in pangenome_obj.seq_collections:
        # print(block.id)
        for seq in block.sequences:
            all_sequences.append(seq)
            
    overlapping_pairs = {}
    for s1, s2 in combinations(all_sequences,2):
        inter_len = 0
        if s1.seq_name != s2.seq_name:
            continue
        inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
        if inter_len > 0:
            # print(s1, s2)
            overlapping_pairs[(s1, s2)] = inter_len
            
    sorted_pairs = sorted(list(overlapping_pairs), key=lambda x: overlapping_pairs[x], reverse=True)
    print(len(sorted_pairs))
    for s1, s2 in sorted_pairs:
        trim_overlap(s1, s2)
        
    overlapping_pairs = {}
    for s1, s2 in combinations(all_sequences,2):
        if s1.seq_name != s2.seq_name:
            continue
        inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
        if inter_len > 0:
            # print(s1, s2)
            overlapping_pairs[(s1, s2)] = inter_len

    print(overlapping_pairs)
    new_blocks_dict = {}
    for seq in all_sequences:
        if seq.cluster_id in new_blocks_dict:
            new_blocks_dict[seq.cluster_id].add(seq)
        else:
            new_blocks_dict[seq.cluster_id] = {seq}

    new_blocks = []
    for block in pangenome_obj.seq_collections:
        if block.id in new_blocks_dict:
            print(block.id)
            block.sequences = new_blocks_dict[block.id]
        new_blocks.append(block)
    pangenome_obj_new = deepcopy(pangenome_obj)
    pangenome_obj_new.seq_collections = new_blocks
    pangenome_obj = pangenome_obj.convert_coords_system(old_coord_sys)
    pangenome_obj_new = pangenome_obj_new.convert_coords_system(old_coord_sys)
    return pangenome_obj_new


        
    
    

In [142]:
maf = maf_parser.parse_maf(maf_dir)
small_maf_0 = maf_parser.MAF(list(maf.seq_collections)[:500])

In [229]:
small_maf = deepcopy(small_maf_0)
small_maf = small_maf.convert_coords_system("gff")

all_sequences = []
# blocks_subset = deepcopy(list(maf.seq_collections)[:500])
for block in small_maf.seq_collections:
    # print(block.id)
    for seq in block.sequences:
        all_sequences.append(seq)
        
overlapping_pairs = {}
for s1, s2 in combinations(all_sequences,2):
    inter_len = 0
    if s1.seq_name != s2.seq_name or s1.strand != s2.strand:
        continue
    inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    if inter_len > 0:
        # print(s1, s2)
        overlapping_pairs[(s1, s2)] = inter_len

small_maf = small_maf.convert_coords_system("maf")

In [245]:
len(overlapping_pairs)

69

In [246]:
trimmed_blocks = []
for seq_pair in overlapping_pairs:
    for s in seq_pair:
        trimmed_blocks.append(s.cluster_id)

In [247]:
trimmed_blocks

[3194,
 3194,
 8199,
 8199,
 1407,
 1407,
 6242,
 6242,
 3113,
 3201,
 3113,
 3201,
 325,
 6218,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 1816,
 982,
 982,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 2448,
 733,
 341,
 341,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 6239,
 2442,
 2442,
 2442,
 2442,
 6232,
 6232,
 6232,
 6232,
 215,
 215]

In [242]:
small_maf = deepcopy(small_maf_0)

maf_trimmed = trimm_overlaps_maf(small_maf)

inter 7
5299_1_3.contig00169: 1148 - 1654, -1
ATGCTGCCGCTGGCGGTGCAGAACAAAAACATCTTTCGCCTGGCTGCCGCTCAGGCGCTGGCCGGCGCCAACTCGGTGGTCTTCTACGCTACCGGCGCGATAGTGGGTAACGCGATTGCCCCCAGCCCGTCGCTGGCCACGCTGCCGATCACCCTGTTTGTGCTGGGGATGGCGGCCTCCATTCTGCCTTTCGGCGCGCTGGCGCGAACGCGGGGCCGCAAAGCGGCGTTCAGGCTCGGCACCGGCGCCGGGATGGTGACCGGACTGGCGGCCGCCCTGGCGGTAGTGCTGAATTCTTTTCTGCTGTTTTGTCTCGCCGCGATGCTTGGCGGCGCCTATGCTGCCGTGGCGCTCAGTTTTCGTTTTGCCGCCACCGACGGCGTCGCCCCGGCGC----------------------------------------------------------------------------------------------------------CGCCCCCCCCCCCCCGGGGCGGGGGCGGG------------------------------------------------------------------GGGGGGGGGGGCCGCTGCGCGAGATTGTCCGCCAGCCGGGTTTTGCCACTACGGTATTCAGCGGCGCGGTGGCCTATATGGTGA-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [248]:
blocks_dict = maf_trimmed.get_seq_collections_dict()
trimmed_block = blocks_dict[trimmed_blocks[0]]

In [249]:
trimmed_block.to_MAF_block(a)

AssertionError: [1180 1180 1180 1180 1180 1180 1180 1180 1180 1180 1180  499],

In [221]:
maf_trimmed.coord_system

'maf'

In [190]:
small_maf = maf_trimmed.convert_coords_system("gff")

all_sequences = []
# blocks_subset = deepcopy(list(maf.seq_collections)[:500])
for block in small_maf.seq_collections:
    # print(block.id)
    for seq in block.sequences:
        all_sequences.append(seq)
        
overlapping_pairs = {}
for s1, s2 in combinations(all_sequences,2):
    inter_len = 0
    if s1.seq_name != s2.seq_name:
        continue
    inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    if inter_len > 0:
        # print(s1, s2)
        overlapping_pairs[(s1, s2)] = inter_len

In [191]:
len(overlapping_pairs)

0

### Also check overlaps in maf coords

In [139]:
small_maf.coord_system

'gff'

In [222]:
small_maf_maf = maf_trimmed

In [223]:
all_sequences = []
# blocks_subset = deepcopy(list(maf.seq_collections)[:500])
for block in small_maf_maf.seq_collections:
    # print(block.id)
    for seq in block.sequences:
        all_sequences.append(seq)
        
overlapping_pairs = {}
for s1, s2 in combinations(all_sequences,2):
    inter_len = 0
    if s1.seq_name != s2.seq_name:
        continue
    if s1.strand != s2.strand:
        continue
    inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    if inter_len > 0:
        # print(s1, s2)
        overlapping_pairs[(s1, s2)] = inter_len

In [224]:
len(overlapping_pairs)

1

In [225]:
overlapping_pairs

{(<pgtools.maf_parser.MAFseq at 0x7f7ea6f85300>,
  <pgtools.maf_parser.MAFseq at 0x7f7ea6f853c0>): 1}

In [226]:
weird_seqs = list(overlapping_pairs.keys())[0]
print(weird_seqs)
s1, s2 = weird_seqs
print(s1)
print(s2)
intersection_len((s1.start, s1.end), (s2.start, s2.end))

(<pgtools.maf_parser.MAFseq object at 0x7f7ea6f85300>, <pgtools.maf_parser.MAFseq object at 0x7f7ea6f853c0>)
5235_6_6.contig00109: 12561 - 13548, -1
5235_6_6.contig00109: 11565 - 12561, -1


1

In [202]:
for s1,s1 in overlapping_pairs.keys():
    print(s1.strand)
    print(s2.strand)

-1
1


In [204]:
all_sequences[-1]

## Now retrieve sequences using gff files

In [ ]:
# def clean_block_by_trimming(block):
#     print(block.id)
#     seqs_to_remove = set()
#     # overlapping_pairs = {}
#     for s1, s2 in combinations(block.sequences,2):
#         if s1.seq_name != s2.seq_name:
#             continue
#         inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
#         if inter_len > 0:
#             # print(s1, s2)
#             # overlapping_pairs[tuple(sorted((s1.cluster_id, s2.cluster_id)))] = inter_len
#             if len(s1) < len(s2):
                
#     return seqs_to_remove


In [112]:
gffs_dir = "/home/pampuch/studia/magisterka/test_data/gffs/klebsiella_subset/gff"

In [113]:
gff_dict = {}
for gff_file in os.listdir(gffs_dir):
    gff_dict[gff_file.split(".")[0]] = parse_gff(os.path.join(gffs_dir,gff_file), store_sequences=True)

In [117]:
scaff_seqs = {}
for genome, gff_obj in gff_dict.items():
    for scaff in gff_obj.scaffolds:
        scaff_seqs[genome + "." + scaff.name] = scaff.seq
        # print(scaff.seq[:30])

In [119]:
blocks_trimmed = maf_trimmed.get_seq_collections_dict()
blocks_0 = small_maf_0.get_seq_collections_dict()

In [125]:
for clust_id in blocks_0.keys():
    # print(blocks_trimmed[clust_id])
    print(len(blocks_0[clust_id]))
    for seq in blocks_0[clust_id].sequences:
        print(seq.seq)
    break

2
---------------------------------------ATGCCTGCAACATCAGGGCGCTGGGCATCTGTTTATCTTGAACCAATGATCGGCTCAGGAGAGCGGCTCACGGTAGCAGTTGCCACGATAACATCATCTGGTGAAATTTTAGTTAAGCCGGCAATCAGAAAAGAAGTTATAGAAGCTATGTATGGCTTCAAAGCACCAGCATTTATCAATGTGGTTGATCTGATTTTGTCGAGTTTGAAGTTGCATCTGGCTGCTAAGGGGGATTTCGTATCATGGCATCCTCCAGTTACAGGCGTCACTATAAGCGCCGTAAGGAATGCTGCGTCATCAAGCCCTGTTGGCATACTCCGCCAAGCGGTATCATTGTCATCGAGTTTGTCATCACTCCTTGAGGCTGAAGAAGATTCCGATGGGTTGCCAGCTAAGCAATCAAGAACTAAAGACAGGTGGCCAATTCAAATCTTTGATGCGGTTATCAGCGCTGACGGACGCAGGGATATTTTTTTCAATAGAAGCTTCACCTTTAGTGATGGACACAGACCGGCTAAAATTTTCTATCTAAGCGATCATGCAGCTATCAACACCGGAAAACTGTTACCCCACAACTTGAATGAGCAGGTTAAGGATGGAAAAGCAAAAATATCTGACTTATCAATGATAAAAAGGCAAGGCGATATTTTTCCAAGAGAAACTCATCAAATGATTATCTACAAACCGGAAGATGATAGCCCTGCATACAATGATAGGCACATAGCATCAATAAATAGTGCGTACCTTTCTTTACAGGACTTGGCTAACACCTACGATGTATCAATCACTTCTGTAAGCACTGCCGAGCATGCAGCAAGGTTAATATTGCAAACTGCAGCATAA
ATGCTAAATCTTGATCAGCTTCTTAAGTCAGCTCCTGCTATGCCTGCAACATCAGGGCGCTGGGCATCTGTTTATCTTGAACCAATGATCGGCTCAGGAGAGCGGCTCACGGTAG

In [132]:
all_sequences = [seq for block in list(small_maf_0.seq_collections)[:500] for seq in block.sequences]
overlapping_pairs = {}
for s1, s2 in combinations(all_sequences,2):
    if s1.seq_name != s2.seq_name or s1.strand != s2.strand:
        continue
    inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    if inter_len > 0:
        # print(s1, s2)
        overlapping_pairs[(s1, s2)] = inter_len

In [133]:
overlapping_pairs

{(<pgtools.maf_parser.MAFseq at 0x7f7ea8789000>,
  <pgtools.maf_parser.MAFseq at 0x7f7ed4505300>): 5,
 (<pgtools.maf_parser.MAFseq at 0x7f7ea8789870>,
  <pgtools.maf_parser.MAFseq at 0x7f7ed4505330>): 5,
 (<pgtools.maf_parser.MAFseq at 0x7f7ea878a0b0>,
  <pgtools.maf_parser.MAFseq at 0x7f7ed4505360>): 5,
 (<pgtools.maf_parser.MAFseq at 0x7f7ea87890c0>,
  <pgtools.maf_parser.MAFseq at 0x7f7ed45053c0>): 5,
 (<pgtools.maf_parser.MAFseq at 0x7f7ea87888b0>,
  <pgtools.maf_parser.MAFseq at 0x7f7ed4505390>): 5,
 (<pgtools.maf_parser.MAFseq at 0x7f7ea8789930>,
  <pgtools.maf_parser.MAFseq at 0x7f7ed4505420>): 5,
 (<pgtools.maf_parser.MAFseq at 0x7f7ea878a170>,
  <pgtools.maf_parser.MAFseq at 0x7f7ed45057e0>): 5,
 (<pgtools.maf_parser.MAFseq at 0x7f7ea878a230>,
  <pgtools.maf_parser.MAFseq at 0x7f7ed4505180>): 5,
 (<pgtools.maf_parser.MAFseq at 0x7f7ea8789360>,
  <pgtools.maf_parser.MAFseq at 0x7f7ed45050f0>): 5,
 (<pgtools.maf_parser.MAFseq at 0x7f7ea878a3b0>,
  <pgtools.maf_parser.MAFseq at 0

In [136]:
for s1, s2 in overlapping_pairs:
    s1_ = deepcopy(s1)
    s2_ = deepcopy(s2)
    trim_overlap(s1_, s2_)
    print(s1)
    print(s1_)
    break

5193_1_5.contig00097: 7304 - 8102, 1
5193_1_5.contig00097: 7309 - 8102, 1


1. FIRST YOU HAVE TO CONVERT TO GFF - HAVE TO CONSIDER BOTH + AND - STRAND
2. if sequences in one block overlap - delete the shorter one
3. if sequences in different blocks overlap - check which overlaping sequence has max overlap
4. PROBLEM: often one block overlaps with multiple other blocks

-> Mabye try different approach: take care of overlaps for each pair of blocks separately, starting from ones with highest overlap

1. Take care of overlaps within blocks: check overlaps only withing each block end delete shorter duplicate
2. Check for overlaps between block. Sort from boggest overlap to shortest.

In [74]:
overlapping_pairs = {}

        
for s1, s2 in combinations(all_sequences,2):
    if s1.seq_name != s2.seq_name or s1.strand != s2.strand:
        continue
    inter_len = intersection_len((s1.start, s1.end), (s2.start, s2.end))
    if inter_len > 0:
        print(s1, s2)
        overlapping_pairs[tuple(sorted((s1.cluster_id, s2.cluster_id)))] = inter_len

5150_3_5.contig00039: 28844 - 29492, 1 5150_3_5.contig00039: 29472 - 30300, 1
5151_2_6.contig00024: 45495 - 46143, 1 5151_2_6.contig00024: 46123 - 46951, 1
5151_6_6.contig00006: 179406 - 180054, -1 5151_6_6.contig00006: 180034 - 180862, -1
5193_1_5.contig00089: 2724 - 3372, 1 5193_1_5.contig00089: 3352 - 4180, 1
5193_2_6.contig00053: 2738 - 3386, -1 5193_2_6.contig00053: 3366 - 4194, -1
5193_8_2.contig00058: 2737 - 3385, 1 5193_8_2.contig00058: 3365 - 4193, 1
5197_2_1.contig00010: 143392 - 144040, 1 5197_2_1.contig00010: 144020 - 144848, 1
5197_7_4.contig00010: 110851 - 111499, -1 5197_7_4.contig00010: 111479 - 112307, -1
5197_7_5.contig00032: 126 - 546, 1 5197_7_5.contig00032: 526 - 1354, 1
5235_5_12.contig00161: 2728 - 3427, 1 5235_5_12.contig00161: 3407 - 4235, 1
5235_6_11.contig00368: 2732 - 3380, 1 5235_6_11.contig00368: 3360 - 4263, 1
5299_1_3.contig00187: 65 - 575, -1 5299_1_3.contig00187: 555 - 1383, -1
5299_7_4.contig00151: 2702 - 3350, -1 5299_7_4.contig00151: 3330 - 4158, -1

In [6]:
# len(list(combinations(all_sequences,2)))

In [80]:
print(overlapping_pairs)

{(2599, 7261): 21, (7728, 7728): 5, (6932, 6932): 54, (660, 7426): 1, (7427, 7428): 5, (6939, 6939): 48, (657, 7978): 5, (2613, 2613): 2, (7747, 7747): 8, (3399, 7971): 9, (473, 473): 59, (6578, 6578): 54, (2490, 2490): 2, (7540, 8024): 5, (3938, 3938): 39, (7977, 7977): 35}


In [76]:
sorted_blocks = list(overlapping_pairs.keys())
sorted_blocks.sort(key = lambda x: overlapping_pairs[x], reverse=True)

In [77]:
sorted_blocks

[(473, 473),
 (6932, 6932),
 (6578, 6578),
 (6939, 6939),
 (3938, 3938),
 (7977, 7977),
 (2599, 7261),
 (3399, 7971),
 (7747, 7747),
 (7728, 7728),
 (7427, 7428),
 (657, 7978),
 (7540, 8024),
 (2613, 2613),
 (2490, 2490),
 (660, 7426)]

In [81]:
for s1, s2 in overlapping_pairs[:10]:
    print("cluster_ids:",s1.cluster_id, s2.cluster_id)
    print(s1)
    print(s2)
    print("======")

TypeError: unhashable type: 'slice'